# Count labels per class


In [1]:
import glob
from collections import defaultdict
import os
import json
import numpy as np
import cv2
from PIL import Image
import torch

In [2]:
#### Process and load annotatinos according to RAPID implementation
def load_anns(img_dir,json_path):
        coco = False
        print(f'Loading annotations {json_path} into memory...')
        with open(json_path, 'r') as f:
            json_data = json.load(f)

        for ann in json_data['annotations']:
            img_id = ann['image_id']
            #print(img_id)
            new_ann = None
            # get width and height 
            if not 'rbbox' in ann:
                # using COCO dataset. 4 = [x1,y1,w,h]
                coco = True
                ann['bbox'][0] = ann['bbox'][0] + ann['bbox'][2] / 2
                ann['bbox'][1] = ann['bbox'][1] + ann['bbox'][3] / 2
                ann['bbox'].append(0)
                if ann['bbox'][2] > ann['bbox'][3]:
                    ann['bbox'][2],ann['bbox'][3] = ann['bbox'][3],ann['bbox'][2]
                    ann['bbox'][4] -= 90
                new_ann = ann['bbox']
            else:
                # using rotated bounding box datasets. 5 = [cx,cy,w,h,angle]
                assert len(ann['rbbox']) == 5, 'Unknown bbox format' # x,y,w,h,a
                new_ann = ann['rbbox']


                if new_ann[2] > new_ann[3]:
                    new_ann[2], new_ann[3] = new_ann[3],new_ann[2]
                    new_ann[4] -= 90 if new_ann[4] > 0 else -90

            if new_ann[2] == new_ann[3]:
                new_ann[3] += 1 # force that w < h

            new_ann[4] = np.clip(new_ann[4], -90.0, 90.0 - 1e-14)

            assert new_ann[2] < new_ann[3]
            assert new_ann[4] >= -90 and new_ann[4] < 90

            # override original bounding box with rotated bounding box
            ann['bbox'] = torch.Tensor(new_ann)
            #print(img_id)
            imgid2anns[img_id].append(ann)

        for img in json_data['images']:
            img_id = img['id']
            #print(img_id)
            assert img_id not in imgid2path
            anns = imgid2anns[img_id]
            # if there is crowd gt, skip this image
            if coco and any(ann['iscrowd'] for ann in anns):
                continue

            img_ids.append(img_id)
            imgid2path[img_id] = os.path.join(img_dir, img['file_name'])
            imgid2info[img['id']] = img

        catids = [cat['id'] for cat in json_data['categories']]

In [3]:
### Store the required values generated during load_anns function
coco =  False
img_ids = []
imgid2info = dict()
imgid2path = dict()
imgid2anns = defaultdict(list)
catids = []

load_anns("/localdata/saurabh/dataset/DST/val/", "/localdata/saurabh/dataset/DST/instances_val_with_rbbox.json" )


Loading annotations /localdata/saurabh/dataset/DST/instances_val_with_rbbox.json into memory...


In [5]:
all_labels = []
for i_d in range(len(imgid2info)):
    annotation = imgid2anns[i_d+1]
    for anno in annotation:
        all_labels += [anno['category_id']]
    
    #break

In [6]:
labels = ['person', 'chair', 'table', 'armchair', 'tv', 'wheeled_walker']
label_count = {}
for lab in range(1,7):
    count = np.where(np.array(all_labels) == lab)[0]
    label_count[labels[lab-1]] = len(count)
    print(len(count))
    
    #break

3482
610
629
625
629
1126


In [7]:
label_count

{'person': 3482,
 'chair': 610,
 'table': 629,
 'armchair': 625,
 'tv': 629,
 'wheeled_walker': 1126}